<a href="https://colab.research.google.com/github/dylandeppiesse/realty-project/blob/main/Realty_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
from requests import get
import time as t
from random import randint
import pandas as pd
import itertools
import re

In [2]:
# setting headers which will make it look like this is coming
# from an actual browser instead of a scraper
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML,\
            like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [3]:
realtor = 'https://www.rew.ca/sitemap/real-estate/bc'
response = get(realtor, headers=headers)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
%%time
#setting up a list to get every sub-region
subregion = []
bc_urls = 'https://www.rew.ca/sitemap/real-estate/bc'
bc_r = get(bc_urls,headers = headers)
bc_html = BeautifulSoup(bc_r.text, 'html.parser')
subregion_containers = bc_html.find_all('div', class_ ='gridblock-item')

CPU times: user 80 ms, sys: 1.04 ms, total: 81.1 ms
Wall time: 701 ms


In [5]:
len(subregion_containers)

177

In [6]:
if subregion_containers != []:
    for container in subregion_containers:
        url = container.find('a', class_= 'gridblock-link')
        subregion.append(url)

t.sleep(randint(1,3))

print('Scraped {} links.'.format(len(subregion)))

Scraped 177 links.


In [7]:
#converting subregions to strings to allow regex 
subregion_str=[str(region) for region in subregion]

#performing regex to remove unecessary info
subregion_urls = []
for item in subregion_str:
    subregion_urls.append(re.findall("href=(.+?)>",item))

In [8]:
#convert to list of STR to perform changes on
subregion_url_str=[str(region) for region in subregion_urls]

#cleaning up subregion_urls so that we can insert them in a loop to get Neigbourhoods
#this loop needs to look like rew.ca + subregion_url
clean_urls = []
for region in subregion_url_str:
    clean_urls.append(region.replace('"','').replace("['",'').replace("']",''))

In [9]:
%%time 
#loop to get neighbourhoods from subregions list above
cities = []
for url in clean_urls:
    subregion_url = 'https://www.rew.ca'+ url
    r = get(subregion_url, headers=headers)
    city_html = BeautifulSoup(r.text, 'html.parser')
    city_container = city_html.find_all('div', class_ = 'gridblock-item')
    
    if city_container != []:
        for container in city_container:
            city = container.find_all('a', class_ = 'gridblock-link')
            cities.append(city)

    t.sleep(randint(1,3))

CPU times: user 8.88 s, sys: 506 ms, total: 9.39 s
Wall time: 8min 12s


In [13]:
print(len(cities))
print(cities[0])

685
[<a class="gridblock-link" href="/sitemap/real-estate/belgo-black-mountain-kelowna-bc">Belgo - Black Mountain
(26)
</a>]


In [ ]:
#we now need to clean up these links, similar to what we did for the subregions

In [12]:
cities[:10]

[[<a class="gridblock-link" href="/sitemap/real-estate/belgo-black-mountain-kelowna-bc">Belgo - Black Mountain
  (26)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/central-city-kelowna-bc">Central City
  (125)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/glenmore-clifton-dilworth-kelowna-bc">Glenmore - Clifton - Dilworth
  (33)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/highway-97-kelowna-bc">Highway 97
  (10)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/mckinley-kelowna-bc">McKinley
  (26)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/north-mission-crawford-kelowna-bc">North Mission - Crawford
  (15)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/rutland-kelowna-bc">Rutland
  (55)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/southeast-kelowna-bc">Southeast Kelowna
  (39)
  </a>],
 [<a class="gridblock-link" href="/sitemap/real-estate/south-pandosy-klo-kelo

In [11]:
# looks like all the neighbourhoods work and are in our listings list, we now need to get the actual listings (I think)